# Simple Debug Notebook: MLFlow Tracing for Claude MCP Agent

This is a simplified version that avoids problematic imports and focuses on core tracing functionality.

In [ ]:
# Simple, robust imports
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Dict, Any, List
import tempfile
import warnings
warnings.filterwarnings('ignore')

# Core MLFlow
import mlflow
from mlflow import MlflowClient

# Our agent
from dotenv import load_dotenv
from agent import create_agent

load_dotenv()
print("✅ All imports successful")

In [ ]:
# Simple MLFlow setup
mlflow_dir = os.path.abspath(os.getenv("MLFLOW_DIR", tempfile.mkdtemp(prefix="mlflow_debug_")))
mlflow_uri = f"file://{mlflow_dir}"

mlflow.set_tracking_uri(mlflow_uri)
experiment_name = "claude_mcp_debug_simple"
mlflow.set_experiment(experiment_name)

print(f"✅ MLFlow setup complete")
print(f"📁 Tracking URI: {mlflow_uri}")
print(f"🧪 Experiment: {experiment_name}")

In [ ]:
# Create agent
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
MCP_SERVER_URL = "http://0.0.0.0:3000/mcp"
MODEL_ID = "claude-3-5-sonnet-20241022"

print("Creating agent...")
agent = create_agent(ANTHROPIC_API_KEY, MCP_SERVER_URL, MODEL_ID)
print(f"✅ Agent created with {len(agent.get_available_tools())} tools")
print(f"Available tools: {agent.get_available_tools()}")

In [ ]:
# Enabling tracing for LangGraph (LangChain)
mlflow.langchain.autolog()

# Test with agent - this should now work without pydantic errors
response = agent.chat("Find me romantic Italian restaurants near Laguna Beach, CA for tonight")
print(f"Claude: {response}")

In [ ]:
# Run test scenarios
print("🧪 Running traced test scenarios\n")

test_threads = [
  [
    "Find me romantic Italian restaurants near Laguna Beach, CA for tonight",
    "Sounds great, can you book a table for two for 7 PM?",
    "The booking will be for 'Arthur Nemo', my number is 123-456-7890"
  ],
  [
    "I need casual Japanese restaurants near Irvine, CA for lunch",
    "What is the price range for the second restaurant?",
    "Can you book a table for four at the second restaurant for 12:30pm?",
    "Book it under the name Vince Van der Snook, phone: 123-456-7890"
  ],
  [
    "What do you think about Taiwanese street food?",
    "Can you recommend a good Taiwanese street food place near Costa Mesa, CA?",
    "Do they offer takeout?",
    "Can you check if they have vegetarian options?",
    "Can I order online?",
  ]
]

for thread_idx, conversation in enumerate(test_threads, 1):
    print(f"\n{'='*60}")
    print(f"Test Thread {thread_idx}:")

    for i, user_input in enumerate(conversation, 1):
        print(f"User Input {i}: {user_input}")
        try:
            response = agent.chat(user_input, f"test_thread_{thread_idx}")
            print(f"Claude Response: {response}")
        except Exception as e:
            print(f"Error: {e}")

print(f"\n🎯 Completed {len(test_threads)} test conversations")